In [84]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from functions import * # Our functions 
import os
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')


In [78]:
def reverse_from_linear_polarity(original_X, item_to_be_reversed):
    x, y, r = original_X.shape[0]/2, original_X.shape[1]/2, np.sqrt(((original_X.shape[0]/2.0)**2.0)+((original_X.shape[1]/2.0)**2.0))
    X_inverse = cv2.linearPolar(item_to_be_reversed, (x, y), r, cv2.WARP_INVERSE_MAP)
    return X_inverse

In [79]:
image_folder = os.getcwd()+'/static/uploads'

In [81]:
def find_circle(filename):
    image = cv2.imread(filename, 0)
    size = np.sqrt(image.size)
    output = image.copy()
    center_x, center_y = None, None
    circles = cv2.HoughCircles(image,cv2.HOUGH_GRADIENT,1,20,param1=50,param2=30,minRadius= int(size*5/12), maxRadius= int(size/2))
    if circles is not None:
        # convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype("int")

        # loop over the (x, y) coordinates and radius of the circles
        for (x, y, r) in circles:
            # draw the circle in the output image, then draw a rectangle
            # corresponding to the center of the circle
            if np.sqrt((x -size/2)**2+(y-size/2)**2) > (size / 8):
                continue
            center_x, center_y = x, y
            cv2.circle(output, (x, y), r, (0, 255, 0), 4)

            cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
            break
        # show the output image
    if center_x == None:
        center_x = int(size/2)
        center_y = int(size/2)
            

        cv2.rectangle(output, (center_x - 5, center_y - 5), (center_x + 5, center_y + 5), (0, 128, 255), -1)
    return center_x, center_y


In [82]:
def polar_to_cartesian(ori_img, pol_img, pol_lines):
    import numpy as np
    row = len(pol_img)
    column = len(pol_img[0])
    t_pi = np.pi*2
    max_radius = np.sqrt(((ori_img.shape[0]/2.0)**2.0)+((ori_img.shape[1]/2.0)**2.0))
    Klin = column / max_radius
    Kangle = row / t_pi
    cart_lines = []
    for line in pol_lines:
        coords=line[0]
        rho_1, phi_1, rho_2, phi_2 = coords[0], coords[1], coords[2], coords[3]
        angleRad1 = phi_1 / Kangle
        magnitude1 = rho_1 / Klin
        x1 = magnitude1 * np.cos(angleRad1)
        y1 =  magnitude1 * np.sin(angleRad1)
        angleRad2 = phi_2 / Kangle
        magnitude2 = rho_2 / Klin
        x2 = magnitude2 * np.cos(angleRad2)
        y2 =  magnitude2 * np.sin(angleRad2)
        cart_lines.append([[int(x1 + ori_img.shape[0]/2), int(y1+ori_img.shape[1]/2),int(x2+ori_img.shape[0]/2), int(y2+ori_img.shape[1]/2)]])
    return cart_lines

In [83]:
def calculate_ratio(ori_img, cart_lines, center_x, center_y):
    original_image = ori_img.copy()
    ratios = []
    for line in cart_lines:
        coords=line[0]
        
        dis1 = np.sqrt((coords[0] - center_x)**2 + (coords[1] - center_y)**2)
        dis2 = np.sqrt((coords[0] - coords[2])**2 + (coords[1] - coords[3])**2)
        ratios.append(dis1/dis2)
        cv2.line(original_image,(coords[0],coords[1]),(coords[2],coords[3]),[255,255,255],3)
        cv2.line(original_image,(center_x,center_y),(coords[0],coords[1]),[0,255,0],3)
    return original_image, np.average(ratios)

In [87]:
ratios = []
for filename in os.listdir(image_folder):
    image_path = image_folder +'/' + filename
    center_x, center_y = find_circle(image_path)
    img = np.asarray(Image.open(image_path))
    polar_img, pol_lines = detect_lines_cy(img)
    if not pol_lines:
        ratios.append(None)
        continue
    cart_lines = polar_to_cartesian(img, polar_img, pol_lines)
    final_image, ratio = calculate_ratio(img, cart_lines, center_x, center_y)
    ratios.append(ratio)

In [90]:
ratio_df =  pd.DataFrame({'file_name': os.listdir(image_folder), 'distance_ratio': ratios})

In [92]:
ratio_df.to_csv('ratio_df')